### **6.7 - Execute critical tasks first, the others after**

¿Qué pasa si tienes algunas tareas críticas en tu data pipeline que quieres ejecutar primero antes que las otras tareas? o ¿qué pasa si tienes algunos DAGs y quieres asegurarte de que tan pronto como algunos recursos estén disponibles, entonces este DAG tenga prioridad sobre los otros DAGs? Bien, para hacer esto necesitas aprovechar el concepto de "Task priority". Veamos qué significa. Imaginemos que para este data pipeline quieres ejecutar primero "extract_partner_netflix", luego la tarea "extract_partner_snowflake" y finalmente quieres ejecutar la tarea "extract_partner_astronomer".  

<center><img src="https://i.postimg.cc/gjPFgpsr/a1403.png"></center>

¿Cómo puede hacerlo? Bueno, una cosa que hay que saber es que cualquier operador en airflow tiene un argumento llamado "priority_weight", y este argumento se establece en "1" por defecto, lo que significa que, si usted tiene algunas tareas en el mismo nivel y utiliza el "Sequential executor" para ejecutar una tarea a la vez, el planificador elegirá al azar una tarea entre todas esas tareas. ¿Por qué? Porque todas tienen el mismo peso de prioridad. Ahora, puedes modificar esto cambiando este argumento, así que veamos cómo hacerlo. En su editor de código, vamos a añadir una nueva key para cada uno de sus partners llamado "priority", así que escriba:

<center><img src="https://i.postimg.cc/SR705P1d/a1404.png"></center>

Así que una cosa que hay que saber es que cuanto mayor sea el número, mayor prioridad tiene la tarea. A continuación, en su bucle donde se crean las tareas "extract", puede añadir otro argumento a su tarea, que es "priority_weight" igual a "details['priority']". Y haciendo esto, si la tarea es "snowflake", entonces la prioridad aquí y por lo tanto el peso de la prioridad será igual a "2". Si es Netflix, será igual a "3" y así sucesivamente. Y como vas a ver, "Netflix" se ejecutará primero, luego "snowflake" y finalmente "astronomer". 

<center><img src="https://i.postimg.cc/y6FzkJ6R/a1405.png"></center>

Guardamos el archivo y vamos a ver que en la UI de Airflow, activamos el toggle del DAG, refrescamos la página y vamos a esperar un poco y como puedes ver, "Netflix" se ejecuta primero

<center><img src="https://i.postimg.cc/sDsCSV1N/a1406.png"></center>

luego "snowflake"

<center><img src="https://i.postimg.cc/pXCbJgFX/a1407.png"></center>

y finalmente "astronomer". Así es exactamente cómo funciona. Es tan sencillo como eso. Sólo tienes que modificar el priority weight de cada una de tus tareas para modificar las prioridades. Ahora, tienes que saber algo, las prioridades de las tareas se evalúan. De nuevo, si esas tareas "snowflake", "netflix" y "astronomer" se ejecutan en el orden definido por las prioridades, primero "Netflix", "snowflake" y finalmente "astronomer" 

<center><img src="https://i.postimg.cc/VkNyGDq9/a1408.png"></center>

eso es porque comparten el mismo pool. Si tienen diferentes pools, entonces no funcionará. Déjame mostrarte esto. Agreguemos otra key llamada "pool" y llamémosla "snowflake", luego otro pool para "netflix" llamado "netflix" y finalmente para "astronomer" un pool llamado "astronomer"

<center><img src="https://i.postimg.cc/3xfsPm1s/a1409.png"></center>

A continuación, es necesario añadir los diferentes pools en función del partner en el bucle, así que aquí escribe:

<center><img src="https://i.postimg.cc/wBSSBzSF/a1410.png"></center>
<center><img src="https://i.postimg.cc/j2TmzXq7/a1411.png"></center>

Por último, es necesario crear los distintos pools en la interfaz de usuario de Airflow. Ve a Admin > Pools, crea un nuevo registro, llámalo "snowflake" con "1" slot 

<center><img src="https://i.postimg.cc/W4DBjzfh/a1412.png"></center>
<center><img src="https://i.postimg.cc/L5V7v1h0/a1413.png"></center>

luego otro pool llamado "netflix" con "1" slot 

<center><img src="https://i.postimg.cc/gkY1jNT9/a1414.png"></center>
<center><img src="https://i.postimg.cc/02cFwDTz/a1415.png"></center>

y otro llamado "astronomer" también. 

<center><img src="https://i.postimg.cc/SxR57j4D/a1416.png"></center>
<center><img src="https://i.postimg.cc/RZYYP9V9/a1417.png"></center>

Y ahora tienes los tres pools, como "astronomer", "netflix" y "snowflake", vamos a ver lo que obtienes

<center><img src="https://i.postimg.cc/JhB2vgGX/a1418.png"></center>

Así que regresa a DAGs, haz click en tu DAG "my_dag", luego Graph View y activa el toggle, refresca la página y esperemos un poco. Y como puedes ver todas las tareas se han ejecutado al mismo tiempo.

<center><img src="https://i.postimg.cc/V6QV8RCV/a1419.png"></center>

No has obtenido las prioridades como antes donde "Netflix" se ejecutaba primero, luego "snowflake" y luego "astronomer". Así que ten en cuenta que las prioridades sólo funcionan si se aplican con el mismo pool. Y por cierto, si disparas tu DAG manualmente, las prioridades de tus tareas no se tendrán en cuenta. Dicho esto, hay una última cosa que necesitas saber. Hay un argumento llamado "wait_rule", que define cómo se calculan los priority weights de tus tareas. Permítanme mostrarles esto. Así, por defecto, tiene tres reglas diferentes, y la primera, que es la que por defecto, es "DOWNSTREAM". Lo que significa es, que tienes todas tus tareas con un peso de prioridad "1-1-1-1" y el peso de una tarea dada es la suma agregada de todas las tareas downstream, lo que significa, que esta tarea tiene el peso de prioridad igual a "1"

<center><img src="https://i.postimg.cc/DZrYyXQy/a1420.png"></center>

pero entonces, esta tiene de hecho un peso de prioridad igual a "2". ¿Por qué? Porque 1 + 1,

<center><img src="https://i.postimg.cc/D07pWw5Q/a1421.png"></center>

y entonces esta tarea tiene un peso de prioridad igual a "3". De nuevo, porque hay una tarea downstream, otra y finalmente más el peso de prioridad actual, que es uno.

<center><img src="https://i.postimg.cc/NGxP3Rfd/a1422.png"></center>

Así que esta tarea también tiene un peso de prioridad de "3". 

<center><img src="https://i.postimg.cc/PJK31nFg/a1423.png"></center>

Así que este es realmente el comportamiento por defecto de Airflow, si ejecutas tu DAG, ejecutarás tus tareas así. Pero es bueno saber que detrás de escena, cada tarea tiene un peso de prioridad de acuerdo a su tarea downstream. Ahora, hay otra regla que es "UPSTREAM", y en ese caso es justo al revés. Así que básicamente, usted todavía tiene el peso de prioridad "1-1-1-1" para cada tarea, que es el caso por defecto. 

<center><img src="https://i.postimg.cc/5NNkbjXb/a1424.png"></center>

Pero esta vez, esta tarea tendrá el peso de prioridad más alto. 

<center><img src="https://i.postimg.cc/m2XpDR9M/a1425.png"></center>

¿Por qué? Porque, de nuevo, tendrá "1 + 1 + 1" que le da "3" para el peso de prioridad de esa tarea. 

<center><img src="https://i.postimg.cc/VkJG0T9h/a1426.png"></center>

Obviamente esta tiene un peso de prioridad igual a "2", 

<center><img src="https://i.postimg.cc/4xj865Bx/a1427.png"></center>

y esas tareas mantienen el peso de prioridad igual a "1". Así que en realidad es justo lo contrario.

<center><img src="https://i.postimg.cc/tgPB2xMt/a1428.png"></center>

¿Por qué esto puede ser útil para ti? Bien, imaginemos que tienes múltiples DagRuns y quieres asegurarte de que este DAG se completa antes de ejecutar la primera tarea del otro DAG. 

<center><img src="https://i.postimg.cc/wvp0PBKK/a1429.png"></center>

Bueno, eso es algo que puedes lograr con la "weight rule upstream". Luego está la última regla de peso, que es "ABSOLUTE", y ésta es extremadamente simple, porque realmente se basa en los pesos de prioridad que definas. Entonces, si le das "1" a esta tarea, luego "3" a esa, luego "2" a esa y "5" a esa, bueno, en ese caso los pesos de prioridad no cambiarán incluso si esta tarea 

<center><img src="https://i.postimg.cc/hj1pqqgX/a1430.png"></center>

tiene más tareas downstream que obviamente esta. No cambiará en absoluto. Airflow mantendrá los pesos de prioridad tal y como están definidos.

<center><img src="https://i.postimg.cc/7hPXHQYm/a1431.png"></center>

¿Y por qué esto puede ser útil para ti? Bueno, porque tú tienes el control total sobre las prioridades de tus tareas. Dicho esto, una última cosa que puedes hacer para asegurarte de que un DAG se prioriza frente a otro DAG, es definir dentro del argumento por defecto de tu DAG, "priority weights" y establecerlo en 99, por ejemplo. Ok, así todas las tareas de tu DAG tendrán un peso de prioridad más alto que el otro DAG que se establecerá en "1" por defecto. Así que, eso es todo acerca de las prioridades, no dudes en jugar con ellas, es realmente una herramienta poderosa y nos vemos para el próximo video.